In [1]:
!pip install requests
!pip install BeautifulSoup4
!pip install tqdm
!pip install natasha
!pip install numpy

In [2]:
from bs4 import BeautifulSoup
import numpy
from tqdm import tqdm
import requests
import numpy
import json
import matplotlib

In [101]:
headers = {
    "User-Agent": "Mozilla/5.0 \
           (Macintosh; Intel Mac OS X 10_10_1) \
           AppleWebKit/537.36 (KHTML, like Gecko) \
           Chrome/39.0.2171.95 Safari/537.36"
}

In [102]:
baseUrl = 'https://career.habr.com/vacancies?type=all'

In [103]:
request = requests.get(baseUrl, headers=headers)
cont = request.content

In [104]:
soup = BeautifulSoup(request.content, 'html.parser')

In [105]:
soup.title

<title>Работа и свежие вакансии для IT специалистов от прямых работодателей — Хабр Карьера</title>

# Task 1

### Fetch at least 100 non-blank carreers


#### Creating links

In [106]:
divis = ['backend','frontend','apps','software','testing','administration','design',
             'management','marketing','analytics','sale','content','support','hr','telecom',
             'other','office','security']
massOfStrings = []

In [107]:
parseLink = 'https://career.habr.com/vacancies?divisions[]={}&type=all'
parseLinkNum = 'https://career.habr.com/vacancies?divisions[]={}&type=all&page={}'

In [108]:
for i in divis:
    massOfStrings.append(parseLink.format(i))

Parsing salaries through all pages and all divisions

In [129]:
salaries = {}

for i in tqdm(divis,'parsing some salaries data...'):
    page = 1
    salaries[i] = []
    while True: # Идея выхода из цикла через reqests.ok для парсинга всех страниц взята у Андрея Савчука

        r = requests.get(parseLinkNum.format(i,str(page)),headers=headers)

        if not r.ok: # Реализация этой идеи
            break

        soup = BeautifulSoup(r.content, 'html.parser')
        vacancyCard = soup.find_all('div', 'vacancy-card')

        if vacancyCard == []:
            break

        for card in vacancyCard:
            salary = card.find('div','vacancy-card__salary').find('div','basic-salary').text
            salaries[i].append(salary)

        # print(vacancyCard)

        # for i in vacancyCard:
        #     print(i.find('div','vacancy-card__salary').find('div','basic-salary').text)
        #

        page += 1

parsing some salaries data...: 100%|██████████| 8/8 [24:25<00:00, 183.16s/it]


In [130]:
safeSal = salaries
safeSal

{'s': ['от 2 500 до 3 500 $',
  '',
  '',
  '',
  '',
  '',
  'от 80 000 до 110 000 ₽',
  'от 120 000 до 150 000 ₽',
  'от 150 000 ₽',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  'До 350 000 ₽',
  '',
  '',
  '',
  '',
  '',
  'от 130 000 до 180 000 ₽',
  'от 220 000 ₽',
  '',
  'от 60 000 до 100 000 ₽',
  'от 2 000 до 3 000 $',
  '',
  'До 150 000 ₽',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  'от 140 000 до 230 000 ₽',
  'от 150 000 до 230 000 ₽',
  'До 250 000 ₽',
  'от 150 000 до 250 000 ₽',
  'До 220 000 ₽',
  'от 75 000 до 100 000 ₽',
  '',
  '',
  '',
  'от 150 000 до 200 000 ₽',
  'от 100 000 до 150 000 ₽',
  'от 200 000 до 300 000 ₽',
  'от 3 200 $',
  '',
  '',
  '',
  '',
  'от 120 000 до 240 000 ₽',
  'от 2 000 $',
  'от 150 000 до 200 000 ₽',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  'от 3 200 $',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
 

# Clean-Up Process 

In [136]:
import re

Getting RUB-USD/RUB-EURO/RUB-TENGE/RUB-YAH

In [137]:
cbrLink = 'https://www.cbr-xml-daily.ru/daily_json.js'
data = requests.get(cbrLink)
df = data.json()['Valute']
usdMultiplier = df['USD']['Value']
euMultiplier = df['EUR']['Value']
uahMultiplier = df['UAH']['Value']
kztMultiplier = df['KZT']['Value']

In [138]:
print(usdMultiplier,euMultiplier,uahMultiplier,kztMultiplier)

71.9882 83.1248 27.3155 16.9245


### Deleting all NaN elements

In [139]:
for divisionSal in safeSal:
    safeSal[divisionSal] = list(filter(lambda a: a != '', safeSal[divisionSal]))

### Dealing with spaces and adding Valute

In [140]:
for divis in safeSal:
    counter = 0
    for line in safeSal[divis]:
        result = re.findall(r'\d+',line)
        result.append(line[len(line)-1])
        if len(result) >= 5:
            minimalSal = int(result[0]+result[1])
            maximalSal = int(result[2]+result[3])
            result = [minimalSal, maximalSal, result[len(result)-1]]
            safeSal[divis][counter] = result
            counter += 1
        else:
            sal = int(result[0]+result[1])
            result = [sal,line[len(line)-1]]
            safeSal[divis][counter] = result
            counter += 1

TypeError: expected string or bytes-like object

### Converting Valutes into Rubles

In [127]:
for divis in safeSal:
    for line in safeSal[divis]:
        if line[len(line)-1] == '$':
            if len(line) == 3:
                line[0] *= usdMultiplier
                line[1] *= usdMultiplier
                line[len(line)-1] = '₽'
            else:
                line[0] *= usdMultiplier
                line[len(line)-1] = '₽'
        elif line[len(line)-1] == '€':
            if len(line) == 3:
                line[0] *= euMultiplier
                line[1] *= euMultiplier
                line[len(line)-1] = '₽'
            else:
                line[0] *= euMultiplier
                line[len(line)-1] = '₽'
        elif line[len(line)-1] == '₴':
            if len(line) == 3:
                line[0] *= uahMultiplier
                line[1] *= uahMultiplier
                line[len(line)-1] = '₽'
            else:
                line[0] *= uahMultiplier
                line[len(line)-1] = '₽'
        elif line[len(line)-1] == '₸':
            if len(line) == 3:
                line[0] *= kztMultiplier
                line[1] *= kztMultiplier
                line[len(line)-1] = '₽'
            else:
                line[0] *= kztMultiplier
                line[len(line)-1] = '₽'
                
        
safeSal['backend']

[[350000, '₽'],
 [130000, 180000, '₽'],
 [140000, 230000, '₽'],
 [250000, '₽'],
 [100000, 150000, '₽'],
 [120000, 240000, '₽'],
 [1193806452.7174306, '₽'],
 [300000, 350000, '₽'],
 [300000, '₽'],
 [200000, '₽'],
 [200000, '₽'],
 [150000, 250000, '₽'],
 [150000, '₽'],
 [150000, '₽'],
 [180000, 350000, '₽'],
 [80000, 150000, '₽'],
 [150000, 300000, '₽'],
 [350000, '₽'],
 [250000, '₽'],
 [200000, 250000, '₽'],
 [300000, 350000, '₽'],
 [150000, 250000, '₽'],
 [125000, '₽'],
 [2984516131.7935767, '₽'],
 [2984516131.7935767, '₽'],
 [2238387098.8451824, '₽'],
 [2238387098.8451824, '₽'],
 [270000, '₽'],
 [80000, 160000, '₽'],
 [170000, 250000, '₽'],
 [300000, 350000, '₽'],
 [110000, '₽'],
 [170000, '₽'],
 [1305725807.65969, 1678790324.1338868, '₽'],
 [80000, 130000, '₽'],
 [250000, 350000, '₽'],
 [100000, 150000, '₽'],
 [140000, 170000, '₽'],
 [200000, 300000, '₽'],
 [150000, '₽'],
 [150000, 250000, '₽'],
 [180000, 240000, '₽'],
 [150000, 250000, '₽'],
 [200000, '₽'],
 [320000, '₽'],
 [170000,